### Standard imports and global variables

In [1]:
import re
import json
from pathlib import Path
from functools import partial
from collections import Counter

import numpy as np
import pandas as pd
from scipy.stats import ttest_rel
import pytrec_eval

from IPython.display import display


QRELS_PATH = "../../data/test/trec_eval/qrels.txt"
RUN_FILE_PATHS = list(Path("../../data/test/trec_eval/").glob("**/run_files/*.txt"))

### Defining functions to extract run parameters

In [2]:
def get_vector_size(index):
    match = re.search("d\d+", index)
    return int(match.group()[1:])


def get_negative_samples(index):
    match = re.search("n\d+", index)
    return int(match.group()[1:])


def get_window_size(index):
    match = re.search("w\d+", index)
    return int(match.group()[1:])


def get_min_count(index):
    match = re.search("mc\d+", index)
    return int(match.group()[2:])


def get_sample(index):
    match = re.search("(?:s)(\d[e.\-\d]*)", index)
    return float(match.group(1))


def get_epochs(index):
    match = re.search("ep\d+", index)
    return int(match.group()[2:])


def get_ratio(index):
    match = re.search("(?:r\()(.*)(?:\))", index)
    return float(match.group(1))

    
def get_stategy(index):
    return index.split(".")[-1]


def filter_irrelevant_columns(df, suffixes=["_100", "_200", "_500", "_1000"]):
    is_irrelevant = lambda x: any([x.endswith(suffix) for suffix in suffixes])
    cols_to_drop = [col for col in df.columns if is_irrelevant(col)]
    return df.drop(columns=cols_to_drop)


def infer_columns(df):
    # Vector size
    df["vector_size"] = df.index.map(get_vector_size)
    df["ns"] = df.index.map(get_negative_samples)
    df["w"] = df.index.map(get_window_size)
    df["epochs"] = df.index.map(get_epochs)
    df["min_count"] = df.index.map(get_min_count)
    df["sample"] = df.index.map(get_sample)
    df["ratio"] = df.index.map(get_ratio)
    df["strategy"] = df.index.map(get_stategy)
    df = filter_irrelevant_columns(df)
    
    return df


def read_qrels(qrels_path=QRELS_PATH):
    with open(QRELS_PATH, "r") as fp:
        qrels = pytrec_eval.parse_qrel(fp)
    return qrels


def read_runfiles(runfile_paths=RUN_FILE_PATHS, pattern=None):
    if pattern:
        relevant_runfiles = [p for p in runfile_paths if pattern in str(p)]
    else:
        relevant_runfiles = runfile_paths
        
    runs = dict()
    
    for run_file in relevant_runfiles:
        run_id = run_file.stem
        with open(run_file, "r") as fp:
            run = pytrec_eval.parse_run(fp)
        runs[run_id] = run   
        
    return runs
    
    
def evaluate(qrels, runs, measures):
    run_results_by_query = dict()
    run_results = dict()
    for run_id, run in runs.items():
        evaluator = pytrec_eval.RelevanceEvaluator(qrels, measures)
        results = evaluator.evaluate(run)       
        mean_results = {measure: np.mean([v[measure] for _, v in results.items()]) 
                        for measure in results["0"].keys()}
        run_results_by_query[run_id] = results
        run_results[run_id] = mean_results

    return run_results_by_query, run_results


### Reading in query relevances

In [3]:
qrels = read_qrels()

### Inspecting results (by run)

In [4]:
# runs = read_runfiles()
runs = read_runfiles(pattern="r(0.8).full")

run_results_by_query, run_results = evaluate(qrels, runs, ["ndcg"])

results_df = infer_columns(pd.DataFrame(run_results).T)
results_df = results_df.sort_values("ndcg", ascending=False)

with pd.option_context("max_rows", None):
    display(results_df.style.format({"sample": "{:.0e}".format, "ratio": "{:.2f}".format}))

### Significance testing

In [5]:
from functools import reduce

get_ndcg = lambda x :[v['ndcg'] for _,v in run_results_by_query[x].items()]

In [6]:
runs_1 = results_df.query("w == 1 & epochs == 20 & vector_size >=100").sort_values(["vector_size", "ns"])
display(runs_1)

ndcg  vector_size  ns  \
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20...  0.757042          100  10   
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20...  0.805743          100  20   
run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20...  0.769964          150  10   
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20...  0.779419          150  20   
run.Doc2Vec(dm-c,d200,n10,w1,mc5,s1e-05,t8,ep20...  0.771025          200  10   
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20...  0.770504          200  20   
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20...  0.754533          300  10   
run.Doc2Vec(dm-c,d300,n20,w1,mc5,s1e-05,t8,ep20...  0.755746          300  20   

                                                    w  epochs  min_count  \
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20...  1      20          5   
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20...  1      20          5   
run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20...  1      20          5   
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20...  1      20          5   
run.Doc2Vec(dm-c,d200,n10,w1,mc5,s1e-05,t8,ep20...  1      20          5   
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20...  1      20          5   
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20...  1      20          5   
run.Doc2Vec(dm-c,d300,n20,w1,mc5,s1e-05,t8,ep20...  1      20          5   

                                                     sample  ratio strategy  
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d200,n10,w1,mc5,s1e-05,t8,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d300,n20,w1,mc5,s1e-05,t8,ep20...  0.00001    0.8     full

In [7]:
runs_2 = results_df.query("w == 3 & epochs==20 & vector_size >=100").sort_values(["vector_size", "ns"])
display(runs_2)

ndcg  vector_size  ns  \
run.Doc2Vec(dm-c,d100,n10,w3,mc5,s1e-05,t4,ep20...  0.732641          100  10   
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20...  0.765107          100  20   
run.Doc2Vec(dm-c,d150,n10,w3,mc5,s1e-05,t4,ep20...  0.744512          150  10   
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep20...  0.768682          150  20   
run.Doc2Vec(dm-c,d200,n10,w3,mc5,s1e-05,t4,ep20...  0.754117          200  10   
run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20...  0.742200          200  20   
run.Doc2Vec(dm-c,d300,n10,w3,mc5,s1e-05,t4,ep20...  0.727446          300  10   
run.Doc2Vec(dm-c,d300,n20,w3,mc5,s1e-05,t8,ep20...  0.719051          300  20   

                                                    w  epochs  min_count  \
run.Doc2Vec(dm-c,d100,n10,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d150,n10,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d200,n10,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20...  3      20          5   
run.Doc2Vec(dm-c,d300,n10,w3,mc5,s1e-05,t4,ep20...  3      20          5   
run.Doc2Vec(dm-c,d300,n20,w3,mc5,s1e-05,t8,ep20...  3      20          5   

                                                     sample  ratio strategy  
run.Doc2Vec(dm-c,d100,n10,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d150,n10,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d200,n10,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d300,n10,w3,mc5,s1e-05,t4,ep20...  0.00001    0.8     full  
run.Doc2Vec(dm-c,d300,n20,w3,mc5,s1e-05,t8,ep20...  0.00001    0.8     full

In [8]:
run_1_ndcgs = reduce(list.__add__, map(get_ndcg, runs_1.index))
run_2_ndcgs = reduce(list.__add__, map(get_ndcg, runs_2.index))
ttest_rel(run_1_ndcgs, run_2_ndcgs, alternative="greater")

Ttest_relResult(statistic=4.089527097388457, pvalue=2.596450354338026e-05)

### Inspecting results (by query)

In [9]:
res_query = run_results_by_query
res = [res_query[run] for run in res_query.keys()]
res_query_df = pd.DataFrame.from_dict(res[0], orient="index")

for i in range(1, len(res)):
    res_query_df += pd.DataFrame.from_dict(res[i], orient="index")

res_query_df = filter_irrelevant_columns(res_query_df / len(res))
res_query_df = res_query_df.sort_values("ndcg")

print("Queries with lowest average ndcg")
display(res_query_df.head(10))
print("Queries with highest average ndcg")
display(res_query_df.tail(10))

lowest_ndcg_doc_ids = res_query_df.head().index

Queries with lowest average ndcg


ndcg
51  0.208762
38  0.331411
21  0.441311
2   0.461816
0   0.465184
42  0.489702
46  0.494537
33  0.503899
32  0.544333
48  0.596614

Queries with highest average ndcg


ndcg
29  0.908684
11  0.911896
5   0.933403
27  0.951624
7   0.958376
49  0.962345
8   0.974205
43  0.977857
10  0.984622
9   0.992311

In [10]:
run = "run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full"

{k:v["ndcg"] for k,v in res_query[run].items()}

{'0': 0.5068311754437849,
 '1': 0.6856657242067933,
 '2': 0.6256370815118074,
 '3': 0.9340483318026336,
 '4': 0.9252273613906197,
 '5': 1.0,
 '6': 0.8597186998521972,
 '7': 1.0,
 '8': 1.0,
 '9': 1.0,
 '10': 1.0,
 '11': 0.9875671027445805,
 '12': 0.8612335585898477,
 '13': 0.5177415653677128,
 '14': 0.86552094046731,
 '15': 0.9446984783037966,
 '16': 0.8439386943148511,
 '17': 0.8275240460841675,
 '18': 0.8339384423491363,
 '19': 0.6602134401050815,
 '20': 0.9635825499105617,
 '21': 0.49851731908415586,
 '22': 0.8447848264584782,
 '23': 0.8965971569260327,
 '24': 0.7990188763760618,
 '25': 0.7603793323315821,
 '26': 0.8631039009858753,
 '27': 1.0,
 '28': 1.0,
 '29': 1.0,
 '30': 0.4780090533311815,
 '31': 0.8414770302980799,
 '32': 0.5547618335894342,
 '33': 0.831871012159447,
 '34': 0.7224242270408039,
 '35': 0.8451211126956059,
 '36': 0.8662120083432444,
 '37': 1.0,
 '38': 0.3408238075324603,
 '39': 0.8631039009858753,
 '40': 0.6713852295568556,
 '41': 1.0,
 '42': 0.8401250222879123,
 

In [11]:
# Unwanted articles from Proof of Concept notebook
articles_of_interest = [0, 1, 13, 17, 18, 19, 21, 24, 25, 26, 30, 31, 38, 39, 40, 41, 45, 46, 47, 48, 49]

best_runs = Counter()
for article_id in articles_of_interest:
    best_result = None
    best_run = list()
    for run_id, results in res_query.items():
        result = results[str(article_id)]["ndcg"]
        if best_result is None:
            best_result = result
            best_run.append(run_id)
        else:
            if result > best_result:
                best_result = result
                best_run = list()
                best_run.append(run_id)
            elif result >= best_result:
                best_run.append(run_id)
                
    print(article_id, "ndcg: ", best_result)
    if isinstance(best_run, list):
        for run in best_run:
            best_runs.update([run])
            print(run)
    else:
        best_runs.update([run])
        print(run)
    print("\n")

0 ndcg:  0.906049259903061
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full


1 ndcg:  0.9922385874580825
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


13 ndcg:  0.9628945312701557
run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full


17 ndcg:  0.9587435808183749
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


18 ndcg:  0.8962613504376156
run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full


19 ndcg:  0.8257319483794759
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


21 ndcg:  0.676412933984772
run.Doc2Vec(dm-c,d300,n10,w4,mc5,s1e-05,t8,ep20).r(0.8).full


24 ndcg:  0.9184429297429952
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full


25 ndcg:  0.9522357846460809
run.Doc2Vec(dm-c,d200,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full


26 ndcg:  0.9293957157305397
run.Doc2Vec(dm-c,d150,n20,w5,mc5,s1e-05,t4,ep20).r(0.8).full


30 ndcg:  0.8188008749685227
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).fu

In [12]:
print(len(articles_of_interest))
best_runs.most_common()

21


[('run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d200,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d300,n10,w4,mc5,s1e-05,t8,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d150,n20,w5,mc5,s1e-05,t4,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d200,n20,w2,mc5,s1e-05,t8,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d300,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20).r(0.8).full', 2),
 ('run.Doc2Vec(dm-c,d100

### Inspecting top 5 runs of each query that does poorly with the main model

In [13]:
# Unwanted articles from Proof of Concept notebook
articles_of_interest = [0, 1, 13, 17, 18, 19, 21, 24, 25, 26, 30, 31, 38, 39, 40, 41, 42, 45, 46, 47, 48, 49]

best_runs = Counter()
for article_id in articles_of_interest:
    article_results = pd.Series({run_id:results[str(article_id)]["ndcg"] 
                                 for run_id, results in res_query.items()})
    print(article_id)
    top_5 = article_results.sort_values(ascending=False).head()
    display(top_5)
    best_runs.update(top_5.index.values)
    print("\n")   
    


0


run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full    0.906049
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full    0.836247
run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full    0.811691
run.Doc2Vec(dm-c,d100,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full    0.792085
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full    0.785205
dtype: float64



1


run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full     0.992239
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full    0.924630
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full     0.884792
run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full     0.859814
run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full     0.854545
dtype: float64



13


...

run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full    1.0
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full    1.0
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full    1.0
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full    1.0
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full     1.0
dtype: float64



48


run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full     0.873673
run.Doc2Vec(dm-c,d100,n20,w5,mc5,s1e-05,t4,ep20).r(0.8).full    0.862656
run.Doc2Vec(dm-c,d150,n10,w5,mc5,s1e-05,t4,ep20).r(0.8).full    0.853316
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20).r(0.8).full    0.846235
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full    0.829153
dtype: float64



49


run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full    1.0
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20).r(0.8).full    1.0
run.Doc2Vec(dm-c,d300,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full    1.0
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full    1.0
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full    1.0
dtype: float64

In [14]:
best_runs.most_common()

[('run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full', 7),
 ('run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full', 6),
 ('run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full', 6),
 ('run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 6),
 ('run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 6),
 ('run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full', 5),
 ('run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full', 5),
 ('run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full', 5),
 ('run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d300,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full', 4),
 ('run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d100,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full', 3),
 ('run.Doc2Vec(dm-c,d200

In [15]:
models_to_look_into = [
    "Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20)",
    "Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40)",
    "Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20)",
    "Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40)",
    "Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20)"
]

In [16]:
# All articles
articles_of_interest = np.arange(52)
best_result = None
best_run = list()
best_runs = Counter()

for article_id in articles_of_interest:
    for run_id, results in res_query.items():
        result = results[str(article_id)]["ndcg"]
        if best_result is None:
            best_result = result
            best_run.append(run_id)
        else:
            if result > best_result:
                best_result = result
                best_run = list()
                best_run.append(run_id)
            elif result == best_result and run_id not in best_run:
                best_run.append(run_id)
                
    print(article_id, "ndcg: ", best_result)
    if isinstance(best_run, list):
        for run in best_run:
            best_runs.update([run])
            print(run)
    else:
        best_runs.update([run])
        print(run)
    print("\n")

0 ndcg:  0.906049259903061
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full


1 ndcg:  0.9922385874580825
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


2 ndcg:  0.9922385874580825
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


3 ndcg:  0.9922385874580825
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


4 ndcg:  0.9922385874580825
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full


5 ndcg:  1.0
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w5,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w5,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).

run.Doc2Vec(dm-c,d150,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n10,w5,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w1,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w4,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n10,w5,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w5,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w5,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w5,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w5,mc5,s1e-05,t8,ep20).r(0.8).full


34 ndcg:  1.0
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2V

run.Doc2Vec(dm-c,d300,n10,w5,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w5,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full
run.Doc2Vec(dm-c,d200,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w2,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20

run.Doc2Vec(dm-c,d100,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w1,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w4,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w2,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d100,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n20,w3,mc5,s1e-05,t8,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d300,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full
run.Doc2Vec(dm-c,d150,n10,w

In [18]:
print(len(articles_of_interest))
for key in sorted(best_runs.keys(), key=best_runs.get, reverse=True):
    print(key, best_runs[key])

52
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full 51
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40).r(0.8).full 48
run.Doc2Vec(dm-c,d300,n10,w1,mc5,s1e-05,t8,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d200,n20,w3,mc5,s1e-05,t8,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d300,n10,w5,mc5,s1e-05,t8,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d200,n20,w5,mc5,s1e-05,t8,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d200,n10,w4,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d150,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full 47
run.Doc2Vec(dm-c,d200,n10,w3,mc5,s1e-05,t4,ep20).r(0.8).full 47
run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full 47
run.Doc2Vec(dm-c,d300,n10,w3,mc5,s1e-0

### Significance testing

In [19]:
def test_significance(run_1=None, run_2=None, measure="ndcg"):
    query_ids = list(qrels.keys())
    first_scores = [run_results_by_query[run_1][query_id][measure] for query_id in query_ids]
    second_scores = [run_results_by_query[run_2][query_id][measure] for query_id in query_ids]
    return ttest_rel(first_scores, second_scores)

**Vector dimensionality (everything else static):**
* `d100` -> `d150` not significant
* `d100` -> `d200`, `d300` significant 

Conclusions this far: best models should have vector dimensionality `d100` or `d150`

**Negative sampling:**
* (for `d100`) `n20` -> `n10` not significant for `w1`, `w2`, `w3`, significant for `w4`, `w5`
* (for `d150`) `n20` -> `n10` not significant for all window sizes except for `w4`

Conclusions this far: best models should have vector dimensionality `d100` and negative sampling value of `n20` with window sizes `w1`, `w2`, `w3`

**Window size:**
* (for `d100`, `n20`) `w1` -> `w2` not significant, `w1` -> `w3`, `w4`, `w5` significant

Conclusions this far: best models should have vector dimensionality `d100` and negative sampling value of `n20` with window sizes `w1`, `w2`

**Epochs**
Only four models were trained with `e40`:
* `Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40)`
* `Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40)`
* `Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40)`
* `Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40)`

Significance tests showed insignificant differences in perfomance between these models and their counterparts trained with 20 epochs

In [20]:
first_run = "run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full"
second_run = "run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full"

# first_run = results_df.index[0]
# second_run = results_df.index[-1]

display(results_df.loc[first_run])
display(results_df.loc[second_run])

test_significance(first_run, second_run)

ndcg           0.792668
vector_size         100
ns                   20
w                     3
epochs               40
min_count             5
sample          0.00001
ratio               0.8
strategy           full
Name: run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full, dtype: object

ndcg           0.765107
vector_size         100
ns                   20
w                     3
epochs               20
min_count             5
sample          0.00001
ratio               0.8
strategy           full
Name: run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full, dtype: object

Ttest_relResult(statistic=1.5276509332384145, pvalue=0.1327780322212468)

In [21]:
results_df.index[0]

'run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40).r(0.8).full'

In [27]:
results_df.iloc[0:9][["vector_size", "ns", "w", "epochs", "ndcg"]]

vector_size  ns  w  \
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40...          100  20  1   
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40...          100  20  2   
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20...          100  20  1   
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20...          100  20  2   
run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20)...           75  20  2   
run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20)...           50  20  1   
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40...          100  20  3   
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20)...           75  20  1   
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20)...           50  20  2   

                                                    epochs      ndcg  
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep40...      40  0.815255  
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep40...      40  0.806648  
run.Doc2Vec(dm-c,d100,n20,w1,mc5,s1e-05,t4,ep20...      20  0.805743  
run.Doc2Vec(dm-c,d100,n20,w2,mc5,s1e-05,t4,ep20...      20  0.797485  
run.Doc2Vec(dm-c,d75,n20,w2,mc5,s1e-05,t4,ep20)...      20  0.794338  
run.Doc2Vec(dm-c,d50,n20,w1,mc5,s1e-05,t4,ep20)...      20  0.794188  
run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep40...      40  0.792668  
run.Doc2Vec(dm-c,d75,n20,w1,mc5,s1e-05,t4,ep20)...      20  0.788650  
run.Doc2Vec(dm-c,d50,n20,w2,mc5,s1e-05,t4,ep20)...      20  0.780151

In [28]:
run_len = len(results_df)
for i in range(1, run_len):
    _, pval = test_significance(results_df.index[0], results_df.index[i], "ndcg")
    if (pval < 0.05):
        status_msg = "{:2}\t{}\t{:.6f}\t{:.8f}".format(i, results_df.index[i],
                                                    results_df.loc[results_df.index[i], "ndcg"], 
                                                    pval)
        print(status_msg)

 9	run.Doc2Vec(dm-c,d150,n20,w1,mc5,s1e-05,t4,ep20).r(0.8).full	0.779419	0.04346893
10	run.Doc2Vec(dm-c,d200,n20,w2,mc5,s1e-05,t8,ep20).r(0.8).full	0.776343	0.01933376
11	run.Doc2Vec(dm-c,d200,n10,w1,mc5,s1e-05,t8,ep20).r(0.8).full	0.771025	0.00557854
12	run.Doc2Vec(dm-c,d200,n20,w1,mc5,s1e-05,t8,ep20).r(0.8).full	0.770504	0.00494734
13	run.Doc2Vec(dm-c,d150,n10,w1,mc5,s1e-05,t4,ep20).r(0.8).full	0.769964	0.00778647
14	run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full	0.768682	0.00784627
15	run.Doc2Vec(dm-c,d150,n20,w3,mc5,s1e-05,t4,ep40).r(0.8).full	0.766806	0.02461818
16	run.Doc2Vec(dm-c,d100,n20,w4,mc5,s1e-05,t4,ep20).r(0.8).full	0.766104	0.03342871
17	run.Doc2Vec(dm-c,d100,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full	0.765967	0.00792416
18	run.Doc2Vec(dm-c,d100,n20,w3,mc5,s1e-05,t4,ep20).r(0.8).full	0.765107	0.01013532
19	run.Doc2Vec(dm-c,d150,n20,w2,mc5,s1e-05,t4,ep20).r(0.8).full	0.764743	0.01047652
20	run.Doc2Vec(dm-c,d200,n10,w2,mc5,s1e-05,t4,ep20).r(0.8).full	0.761220	0.0

In [ ]:
run_results_by_query[results_df.index[0]]

In [ ]:
run_results_by_query[results_df.index[12]]

In [ ]:
challenger_model = pd.DataFrame.from_dict(run_results_by_query[results_df.index[12]], orient="index")
top_model = pd.DataFrame.from_dict(run_results_by_query[results_df.index[0]], orient="index")

In [ ]:
measure_diffs = top_model - challenger_model

display(measure_diffs)

In [ ]:
display(measure_diffs.loc[lowest_ndcg_doc_ids])

In [ ]:
measure_diffs.loc[lowest_ndcg_doc_ids].mean()

In [ ]:
ttest_1samp(measure_diffs.loc[lowest_ndcg_doc_ids].map, 0)